In [1]:
import sys
sys.path.append('..')
from typing import Dict, List
from copy import deepcopy

In [2]:
import pandas as pd
from mars.movement import *
import numpy as np

In [3]:
from mars.action import Action, ActionType
import mars.serializer as ms
from collections import namedtuple

In [4]:
from pymongo import MongoClient
from bson import ObjectId

In [5]:

def get_point_from_row(df_row_data):

    vector = np.array([
        df_row_data.x_j1,
        df_row_data.y_j2,
        df_row_data.z_j3,
        df_row_data.w_j4,
        df_row_data.p_j5,
        df_row_data.r_j6
    ])

    e1 = df_row_data.e1
    speed = df_row_data.speed
    cnt = df_row_data.cnt
    path = df_row_data.path
    pos_type = df_row_data.position_type

    if pos_type == "JOINT":
        position = PositionJoint(vector, e1)
    else:
        wrist = WristConfig.FLIP if df_row_data.wrist =='F' else WristConfig.NOFLIP
        forearm = ForeArmConfig.UP if df_row_data.forearm == 'U' else ForeArmConfig.DOWN
        arm = ArmConfig.TOWARD if df_row_data.arm =='T' else ArmConfig.BACKWARD

        config = Configuration(wrist, forearm, arm)
        position = PositionCrt(vector, e1, config)

    path = Path[df_row_data.path]

    return Point(cnt, speed, path, position)


"""
def serialize_for_mongo(action, up_dependences_ids, down_dependences_ids):
    
    action_doc = action.to_dict()
    del(action_doc['id'])

    if up_dependences_ids:
        action_doc['upstream_dependences'] = up_dependences_ids

    if down_dependences_ids:
        action_doc['downstream_dependences'] = down_dependences_ids

    return action_doc
"""

def save_in_mongo(action_dict: Dict or List[Dict]):
    
    c = MongoClient()
    carrier = c.get_database('mars').get_collection('carrier')

    if type(action_dict) == list:
        res = carrier.insert_many(action_dict)
    else:
        res = carrier.insert_one(action_dict)

    if not res.acknowledged :
        raise Exception("Error during insertion")
        
    else :
        if type(action_dict) == list:
            print('insertion success, document id : ', str(res.inserted_ids))
            return res.inserted_ids
        else :
            print('insertion success, document id : ', str(res.inserted_id))
            return res.inserted_id

    """
    if type(action) == list:
        actions_docs = []
        for a in action:
            if type(a) == tuple:
                action_doc = serialize_for_mongo(a[0], up_dependences_ids, down_dependences_ids)
                action_doc.update(a[1])
            else:
                action_doc = serialize_for_mongo(a, up_dependences_ids, down_dependences_ids)
            
            actions_docs.append(action_doc)

        res = carrier.insert_many(actions_docs)

        if not res.acknowledged :
            print('Exception during insertion')
            return None
        else :
            print('insertion success, document id : ', str(res.inserted_ids))
            return res.inserted_ids
    
    else:
        if type(action) == tuple:
            action_doc = serialize_for_mongo(action[0], up_dependences_ids, down_dependences_ids)
            action_doc.update(action[1])
        else:
            action_doc = serialize_for_mongo(action, up_dependences_ids, down_dependences_ids)
""" 

def get_action_type(action: str) :
    if(action == 'station'):
        return ActionType['MOVE']['STATION']['WORK']
    elif (action == 'work'):
        return ActionType['MOVE']['ARM']['WORK']
    elif (action == 'approach'):
        return ActionType['MOVE']['ARM']['APPROACH']
    elif (action == 'clearance'):
        return ActionType['MOVE']['ARM']['CLEARANCE']
    elif (action == 'home'):
        return ActionType['MOVE']['STATION']['HOME']
    else:
        raise Exception("default on action type")

def get_action_from_df(action_type: str,
                       description: str,
                       dataframe: pd.DataFrame) -> Action :
    points: List[Point] = []

    for line in dataframe.iterrows():
        points.append(get_point_from_row(line[1]))
    
    ut_uf = dataframe[['UT','UF', 'work_order']].drop_duplicates()
    ut, uf, work_order = int(ut_uf.iloc[0].UT),\
                         int(ut_uf.iloc[0].UF),\
                         int(ut_uf.iloc[0].work_order) if not pd.isna(ut_uf.iloc[0].work_order) else None

    movement = Movement(uf, ut, points)
    
    return Action(id='',
                  atype = get_action_type(action_type),
                  definition = movement,
                  description = description,
                  work_order=work_order)

def get_additionnal_info(dataframe: pd.DataFrame, infolist: List[str]):
    add_info_df = dataframe[infolist].drop_duplicates()
    add_list = [(name, add_info_df.iloc[0][name]) for name in infolist]
    add_list = [(name, convert_np(val)) for name, val in add_list]
        
    return dict(add_list)

def convert_np(value):
    if type(value) == np.int64:
        return int(value)
    elif type(value) == np.float64:
        return float(value)
    else:
        return value
    

In [6]:
df = pd.read_csv('./data/C35_web_5mm.csv')

In [7]:
# return home and go home action
df_home = df[df.mvt =='home']
df_station = df[df.mvt == "station"]
df_approach = df[df.mvt == "approach"]
df_clearance = df[df.mvt == "clearance"]
df_work = df[df.mvt == "work"]

rails = [1, 2, 3, 4, 5, 6]

return_home = get_action_from_df('home', 'return on home position', df_home)
go_home = get_action_from_df('home', 'go on home position', df_home)

return_home = return_home.to_dict()
del return_home['_id']
go_home = go_home.to_dict()
del go_home['_id']

rh_id = save_in_mongo(return_home)
go_home['downstream_dependences'] = [rh_id]
go_id = save_in_mongo(go_home)

for rail in rails: 

  # prepare and insert station mvt in mongo
  df_temp = df_station[df_station.rail==rail]
  station = get_action_from_df('station',
                              'station movement on rail {rail}'.format(rail=rail),
                              df_temp)
  station = station.to_dict()
  add_info = get_additionnal_info(df_temp, ['designation', 'reference', 'id'])

  del station['_id']
  station['upstream_dependences'] = [go_id]
  station['product_reference'] = {
    "designation": add_info['designation'],
    "reference": add_info['reference'],
    "id": add_info['id']
  }

  # station['work_order'] = add_info['work_order']

  station_id = save_in_mongo(station)

  # prepare and insert clearance mvt in mongo
  df_temp = df_clearance[df_clearance.rail==rail]
  clearance = get_action_from_df('clearance',
                              'clearance movement from rail {rail}'.format(rail=rail),
                              df_temp)
  clearance = clearance.to_dict()
  del clearance['_id']
  clearance['product_reference'] = deepcopy(station['product_reference'])

  clearance_id = save_in_mongo(clearance)

  # prepare and insert approach mvt in mongo
  df_temp = df_approach[df_approach.rail==rail]
  approach = get_action_from_df('approach',
                              'approach movement to rail {rail}'.format(rail=rail),
                              df_temp)

  approach = approach.to_dict()
  del approach['_id']
  approach['product_reference'] = deepcopy(station['product_reference'])
  approach['upstream_dependences'] = [station_id]
  approach['downstream_dependences'] = [clearance_id]
  # approach['work_order']= station['work_order']

  approach_id = save_in_mongo(approach)

  # prepare and insert work mvt in mongo
  df_temp = df_work[df_work.rail==rail]
  w_actions = []
  id_list = [item[1] for item in df_temp.id.iteritems()]

  for id in id_list :
    df_w = df_temp[df_temp.id == id]
    add_info = get_additionnal_info(df_w, ['designation', 'reference', 'id'])
    wa = get_action_from_df('work',
        'work movement on location of fastener {ref}.{id} of rail {rail}'
        .format(ref=add_info['reference'], id=add_info['id'], rail=rail),
        df_w)
    wa = wa.to_dict()
    del wa['_id']
    wa['product_reference'] = {
      "designation": add_info['designation'],
      "reference": add_info['reference'],
      "id": add_info['id'],
      "parent": deepcopy(station['product_reference'])
    }
    # wa['work_order'] = add_info['work_order']
    wa['upstream_dependences'] = [approach_id]
    w_actions.append(wa)

  work_ids = save_in_mongo(w_actions)



insertion success, document id :  6172bedeac1e98d6248d5cc2
insertion success, document id :  6172bedeac1e98d6248d5cc4
insertion success, document id :  6172bedeac1e98d6248d5cc6
insertion success, document id :  6172bedeac1e98d6248d5cc8
insertion success, document id :  6172bedeac1e98d6248d5cca
insertion success, document id :  [ObjectId('6172bedeac1e98d6248d5ccc'), ObjectId('6172bedeac1e98d6248d5ccd'), ObjectId('6172bedeac1e98d6248d5cce'), ObjectId('6172bedeac1e98d6248d5ccf'), ObjectId('6172bedeac1e98d6248d5cd0'), ObjectId('6172bedeac1e98d6248d5cd1'), ObjectId('6172bedeac1e98d6248d5cd2'), ObjectId('6172bedeac1e98d6248d5cd3'), ObjectId('6172bedeac1e98d6248d5cd4'), ObjectId('6172bedeac1e98d6248d5cd5'), ObjectId('6172bedeac1e98d6248d5cd6'), ObjectId('6172bedeac1e98d6248d5cd7'), ObjectId('6172bedeac1e98d6248d5cd8'), ObjectId('6172bedeac1e98d6248d5cd9'), ObjectId('6172bedeac1e98d6248d5cda'), ObjectId('6172bedeac1e98d6248d5cdb'), ObjectId('6172bedeac1e98d6248d5cdc'), ObjectId('6172bedeac1e98

In [38]:
station

{'_id': '',
 'definition': {'points': [{'cnt': 0,
    'path': 'JOINT',
    'position': {'config': None,
     'e1': 2250.0,
     'type': 'JOINT',
     'uf': 0,
     'ut': 0,
     'vector': {'j1': 0.0,
      'j2': 18.51,
      'j3': -39.04,
      'j4': 0.0,
      'j5': 106.04,
      'j6': 90.0}},
    'speed': 75}],
  'uf': 1,
  'ut': 3},
 'description': 'station movement on rail 1',
 'downstream_dependences': [],
 'product_reference': {'designation': 'rail',
  'id': 1,
  'reference': 'E53233044200'},
 'type': 'MOVE.STATION.WORK',
 'upstream_dependences': [ObjectId('617276243ef791d4eea93cc5')],
 'work_order': 6}

In [33]:
get_additionnal_info(df_temp, )

{'designation': 'rail', 'id': 1, 'reference': 'E53233044200', 'work_order': 6}